In [1]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

2.2.1
True
True


In [2]:
import flash

/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/flash/__init__.py:21: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/flash/__init__.py:21: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):


In [3]:
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

In [4]:
print('### download IMDb data to local folder')
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")

### download IMDb data to local folder


In [5]:
datamodule = TextClassificationData.from_csv(
    "review",
    "sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    batch_size=4,
)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
print('### define a text classifier ')
classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", labels=datamodule.labels, num_classes=datamodule.num_classes)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).


### define a text classifier 


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# notice the difference here to use MPS instead of CUDA
print('### define the trainer')
trainer = flash.Trainer(max_epochs=4, accelerator="mps", devices=1)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### define the trainer


In [8]:
print('### fine tune the pretrained model to get a new model for sentiment classification')
trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")

### fine tune the pretrained model to get a new model for sentiment classification



  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.


In [9]:
print('### get prediction outputs for two sample sentences')
datamodule = TextClassificationData.from_lists(
    predict_data=[
        "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
        "The worst movie in the history of cinema.",
        "I come from Bulgaria where it 's almost impossible to have a tornado.",
    ],
    batch_size=4,
)
predictions = trainer.predict(classifier_model, datamodule=datamodule, output="labels")
print(predictions)

#print('### get classifier test results')
#trainer.test()


### get prediction outputs for two sample sentences


Predicting: 5625it [00:00, ?it/s]

[['negative', 'positive', 'negative']]


In [12]:
# This does not work. need to figure out why
#trainer.test()

ValueError: An invalid dataloader was passed to `Trainer.test(dataloaders=...)`. Either pass the dataloader to the `.test()` method OR implement `def test_dataloader(self):` in your LightningModule/LightningDataModule.